In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader,TensorDataset

In [7]:
device= torch.device ('cuda' if torch.cuda.is_available() else 'cpu')

This code follows the architecture laid down in Baño-Medina et al.,2020 for CNN10 architecture for temperature downscaling 

Function:  standardising the input temperature data



In [10]:
def standardise_temperature_data(X):

    scaler= StandardScaler() #Ensures all input variables have the same scale
    X_scaled= scaler.fit_transform(X.reshape(-1, X.shape[-1]).reshape(X.shape))
    return X_scaled, scaler

The Temperature Downscaling CNN Model

In [11]:
class TempDownscalingCNN(nn.Module):
    '''Defines a Pytorch neural network model for temperature downscaling based on number of input predictors variables (5 in this case on four different pressure levels)
    and the output size (1 in this case, corr to one daily temperature value at each grid point)'''
    def __init__(self,input_channels,output_size):
        super(TempDownscalingCNN,self).__init__()

        #The convolutional layers : There are three in this architecture following CNN10
        self.conv1= nn.Conv2d(input_channels, 50, kernel_size=3) #50 is the number of kernels, 3x3x20 (20 =5 variables, 4 pressure levels)
        self.conv2= nn.Conv2d(50,25,kernel_size=3) #Dimensions of the kernel are 3x3x50, based on the output from the last layer
        self.conv3= nn.Conv2d(25,10,kernel_size=3) #3x3x25, as the previous layer had 25 kernels

        #So the last convo layer above has 10 kernels of size 3x3. The output from this layer is then flattened and fed into the fully connected layers
        #The fully connected layers 
        self.flatten= nn.Flatten() #The output from the last convo layer is flattened to be fed into the dense layer. Fully connected dense layers in 
        #Pytorch are implemented as Linear Layers (expect a 1 D input)
    
        #Now the dense layers. 
        
        self.fc1= nn.Linear(10 *(output_size-6)* (output_size-6) ,output_size)

        self.fc2 = nn.Linear(10 * (output_size-6) * (output_size-6), output_size)

    #Now, defining the forward pass

    def forward(self,x):
        '''Defining the forward pass of the Temperature Downscaling Model'''
        x=F.relu(self.conv1(x)) #Applying first convolution and activation
        x=F.relu(self.conv2(x)) #Applying second convolution and activation
        x=F.relu(self.conv3(x)) #Applying the third convolution and activation
        x=self.flatten(x) #Flattening the output from the last convolutional layer
        l51=self.fc1(x) #Applying the first fully connected layer
        l52=self.fc2(x) #Applying the second fully connected layer
        
        
        return torch.cat((l51,l52),dim=1)  #Concatenating the output from the last two fully connected layers, in case of temperature downscaling
    
    #Defining the loss function
    
    def loss (pred,target):
        '''Defining the loss function for the model. In this case, the MSE loss is used, also called Gaussian loss, suitable for temperature'''

        mean_pred= pred[:, :target.size(1)] #The first half of the output is the mean

        return F.mse_loss(mean_pred,target)
    

    #Writing the training loop

    def train_model(model, dataloader, criterion, optimizer, num_epochs=10000, device=device, patience=30):
        #Patience : to prevent overfitting in the model, stops the training if loss doesnt improve after 30 epochs
        '''Function to train the temperature downscaling model'''

        model.to(device)

        #The training loop
        best_loss=float('inf')
        patience_counter=0


        for epoch in range (num_epochs):
            model.train()
            running_loss=0.0
            for inputs, targets in dataloader:
                inputs, targets=inputs.to(device), targets.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss=criterion(outputs,targets)
                loss.backward()
                optimizer.step()
                running_loss+=loss.item()*input_size(0)

            epoch_loss= running_loss/len(dataloader.dataset)

#Implementing early stopping to prevent overfitting
            if epoch_loss< best_loss:
                best_loss=epoch_loss
                patience_counter=0

                #Saving the best model
                torch.save(model.state_dict(), 'best_model_temp_downscaling.pth')

            else:
                patience_counter+=1

            if patience_counter>patience:
                print(f'Early stopping at epoch {epoch+1} with best loss {best_loss}')
                break

Making predictions

In [12]:
def predictions(model,dataloader):
    '''Function to make downscaled predictions using the trained model'''
    model.eval()

    predictions=[]
    with torch.no_grad():
        for inputs,targets in dataloader:
            outputs=model(inputs.to(device))
            mean_pred=outputs[:,:inputs.size(1)]
            predictions.append(mean_pred.cpu())
    return torch.cat(predictions,dim=0)
    

EXAMPLE USAGE

In [9]:
#Training the model and validating within the historical period on 20 percent of the data

X_scaled, scaler = standardise_temperature_data(X) # Using the standardisation function to normalise the input data

X_train,X_val,y_train,y_val=train_test_split(X_scaled, y, test_size=0.1,random_state=42) #Doing a 90:10 training :validation split

#Converting the input data into pytorch tensors

train_data= TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
val_data= TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val,dtype=torch.float32))



NameError: name 'X' is not defined

Inputting the data via the channels

In [ ]:
input_channels=X.shape[5] #Number of input variables , geopotential height, zonal wind, meridional wind, specific humidity and temperature
output_size= y.shape[1] # Temperature at each grid point for the entire spatial domain
temperature_downscaling_model_V01= TempDownscalingCNN(input_channels,output_size).to(device)

NameError: name 'X' is not defined

#Training 

In [ ]:
optimizer= optim.Adam(temperature_downscaling_model_V01.parameters(), lr=0.001)
criterion= nn.MSELoss()

train_model(temperature_downscaling_model_V01, train_loader, criterion, optimizer)